<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Valid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model


In [1]:
# !pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/MMLearning/data/Models/Model-03/checkpoint-48000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/MMLearning/data/Models/Model-03/checkpoint-48000")

In [4]:
# image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

In [5]:
model.config

ViTConfig {
  "_name_or_path": "/content/MMLearning/data/Models/Model-03/checkpoint-48000",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder",
    "25": "club_sandwich",
    "26": "crab_cakes",
    "27": "creme_

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [6]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test", split="train")

Resolving data files:   0%|          | 0/45432 [00:00<?, ?it/s]

In [7]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 45432
})

In [8]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.jpg")

In [9]:
ds = ds.filter(filter_funtion)

In [10]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [11]:
for i in range(0,10):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

File Name: apple_pie_103_3.jpg
File Name: apple_pie_105_3.jpg
File Name: apple_pie_107_3.jpg
File Name: apple_pie_10_3.jpg
File Name: apple_pie_119_3.jpg
File Name: apple_pie_123_3.jpg
File Name: apple_pie_124_3.jpg
File Name: apple_pie_12_3.jpg
File Name: apple_pie_132_3.jpg
File Name: apple_pie_135_3.jpg


# Partial Missing Modality Experiment

In [ ]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [ ]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/MMLearning/data/food-101/multimodal/test", split="train")

In [ ]:
ds_f

In [ ]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.1

In [ ]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

In [ ]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

In [ ]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [ ]:
# selected_indices

In [ ]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [ ]:
# selected_indices

In [ ]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [ ]:
fused_ds

In [ ]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test", split="train")

In [ ]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.jpg")

In [ ]:
ds_imgs = ds_joint.filter(filter_funtion)

In [ ]:
ds_imgs

In [ ]:
for i in range(0,5):
    sample = ds_imgs[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

In [ ]:
all_indices = range(len(ds_imgs))
print(all_indices)

In [ ]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [ ]:
# remaining_indices

In [ ]:
missing_ds = ds_imgs.select(remaining_indices)

In [ ]:
missing_ds

In [ ]:
# Combine Both Datasets

In [ ]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [ ]:
ds

# Pre-Processing and Final Validation Loop

In [12]:
labels = ds.features["label"]
labels

ClassLabel(names=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'mac

In [13]:
labels.int2str(ds[532]["label"])

'baklava'

In [14]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [15]:
dataset = ds.with_transform(transform)

In [16]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [17]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [18]:
labels = ds.features["label"].names
labels

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [19]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [20]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [21]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [22]:
batch_size = 32

In [23]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [24]:
model = model.to(device)
model.eval()
predictions, labels = [], []
# valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    # loss = outputs.loss
    # valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [26]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.6260785349533369
precision:  0.6756548094551401
f1_score:  0.6346297193067472
recall 0.6260785349533369


In [27]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.43      0.64      0.51       234
           1       0.69      0.78      0.73       221
           2       0.64      0.81      0.71       226
           3       0.52      0.62      0.57       222
           4       0.73      0.38      0.50       225
           5       0.58      0.82      0.68       224
           6       0.46      0.75      0.57       224
           7       0.57      0.67      0.62       225
           8       0.76      0.35      0.48       226
           9       0.58      0.64      0.61       214
          10       0.67      0.78      0.72       231
          11       0.73      0.82      0.77       227
          12       0.68      0.59      0.63       230
          13       0.87      0.68      0.76       220
          14       0.44      0.59      0.51       231
          15       0.78      0.48      0.60       227
          16       0.51      0.54      0.52       224
          17       0.73    

In [28]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[150   1  10 ...   0   0   4]
 [  0 172   0 ...   0   1   0]
 [  5   0 183 ...   0   0   0]
 ...
 [  3   0   2 ... 157   0   1]
 [  1   1   1 ...   0 111   0]
 [  2   0   0 ...   0   0 181]]


# Joint Representation Experiment


In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.jpg")

In [ ]:
ds_3 = ds.filter(filter_funtion)

In [ ]:
ds_3

In [ ]:
for i in range(0,5):
    sample = ds_3[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

Image Dataset

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return not filename.endswith("_3.jpg")

In [ ]:
ds_4 = ds.filter(filter_funtion)

In [ ]:
ds_4

In [ ]:
for i in range(0,5):
    sample = ds_4[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

Pre-processing

In [ ]:
labels = ds.features["label"]
labels

In [ ]:
labels.int2str(ds[532]["label"])

In [ ]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [ ]:
dataset_3 = ds_3.with_transform(transform)
dataset_4 = ds_4.with_transform(transform)

In [ ]:
dataset_3

In [ ]:
dataset_4

In [ ]:
for item in dataset_4:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

In [ ]:
labels = ds.features["label"].names
labels

In [ ]:
num_of_classes = len(labels)
print(num_of_classes)

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

In [ ]:
valid_dataset_loader_4 = DataLoader(dataset_4, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(valid_dataset_loader_4))

In [ ]:
valid_dataset_loader_3 = DataLoader(dataset_3, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(valid_dataset_loader_3))

In [ ]:
num_batches = len(valid_dataset_loader_3)
num_batches_with_logits_3 = int(num_batches * 1)  # ___% of batches that you want to include (30 for now)
print("Total Batches: ", num_batches)
print("Missing Modaility Batches: ", num_batches_with_logits_3)

In [ ]:
import random

controller = random.sample(range(0, num_batches), num_batches_with_logits_3)
controller = sorted(controller)
print(len(controller))

Validation Loops

In [ ]:
model = model.to(device)
model.eval()
predictions_4, labels_4 = [], []
for batch in valid_dataset_loader_4:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    logits = outputs.logits.detach().cpu()
    probabilities = torch.softmax(logits, dim=1)

    predictions_4.extend(probabilities.cpu().numpy())
    labels_4.extend(label_ids.cpu().numpy())

In [ ]:
predictions_3 = []
batch_idx = 0
for batch in valid_dataset_loader_3:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    # if batch_idx < num_batches_with_logits_3:
    if batch_idx in controller:
        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()
    else:
        length = len(label_ids)
        logits = torch.zeros((length, num_of_classes), device="cpu")

    probabilities = torch.softmax(logits, dim=1)
    predictions_3.extend(probabilities.cpu().numpy())
    batch_idx = batch_idx + 1

In [ ]:
import numpy as np
predictions_3 = np.array(predictions_3)
predictions_4 = np.array(predictions_4)

In [ ]:
predictions_avg = (predictions_3 + predictions_4) / 2
predictions = np.argmax(predictions_avg, axis=1)

In [ ]:
predictions = np.array(predictions)
labels = np.array(labels_4)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [ ]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("recall", recall)
# print("AUROC_score: ", AUROC_score)

In [ ]:
print(classification_report(labels, predictions))

In [ ]:
cm = confusion_matrix(labels, predictions)
print(cm)